In [1]:
import molsysmt as msm
import numpy as np
from molsysmt import pyunitwizard as puw

In [12]:
molsys = msm.convert(msm.systems['pentalanine']['traj_pentalanine.h5'], to_form='molsysmt.MolSys')
d = msm.structure.get_distances(molsys, selection=8, selection_2=18, structure_indices=2000)
print(d[0,0,0])
check_distance = np.isclose(puw.get_value(d[0,0,0], to_unit='nm'), 0.38743175)

0.3874317590975941 nanometer
